In [1]:
import math
import requests
import json
import re
import time, datetime
import numpy as np
import pandas as pd
import gensim
import rdflib
import cProfile, pstats
import spacy
import xml.etree.ElementTree as ET
from owlready2 import *
from urllib.request import urlopen
from neo4j import GraphDatabase
from urllib import parse
from gensim.models import KeyedVectors
from nltk.corpus import wordnet as wn
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# Import google gensim model
filename = "file://C:/Users/Kez/OneDrive/Documents/Jacobs/Yr 3/Thesis/Neo4j_Ontology/GoogleNews.bin.gz" 
model = KeyedVectors.load_word2vec_format(filename, binary=True)

### General Comments
- Relevant packages to be downloaded: owlready2, neo4j, gensim, nltk (run nltk.download() in your environment), rdflib, spacy, snakeviz, scikit-learn
- Download GoogleNews model here https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
- All places with #* are above lines that should be un-commented, in case input ontology is already populated
- Sample input for users can be found in the cells titled "Sample input", above the cell where the function is called

In [3]:
# Specific to Strom-DSI ontology only: creating classes, adding object properties and saving to new file
onto = get_ontology("http://test.org/dsi.owl")

with onto:
    class Company(Thing):
        pass

    class Location(Thing):
        pass
    class Zip(Location):
        pass
    class City(Location):
        pass

    class PowerIndex(Thing):
        pass
    class Dsi365(PowerIndex):
        pass
    class Dsi(PowerIndex):
        pass

    class Timestamp(Thing):
        pass
    class Day(Timestamp):
        pass
    class StartTime(Timestamp):
        pass
    class EndTime(Timestamp):
        pass
        
    class has_for_day(ObjectProperty):
        domain = [onto.Company]
        range = [onto.Day]
        
    class has_for_zip(ObjectProperty):
        domain = [onto.Company]
        range = [onto.Zip]

    class has_for_city(ObjectProperty):
        domain = [onto.Company]
        range = [onto.City] 
        
    class has_for_dsi365(ObjectProperty):
        domain = [onto.Day]
        range = [onto.Dsi365]
        
    class has_for_timestamp(ObjectProperty):
        domain = [onto.Day]
        range = [onto.Timestamp]

    class has_for_start(ObjectProperty):
        domain = [onto.Day]
        range = [onto.StartTime] 
        
    class has_for_dsi(ObjectProperty):
        domain = [onto.Timestamp]
        range = [onto.Dsi] 
        
onto.save(file = "C:/Users/Kez/OneDrive/Documents/Jacobs/Yr 3/Thesis/Neo4j_Ontology/Strom")

In [4]:
# Update url to get data from now-10 mins to now, if necessary
def urlupdate(url):
    # Split url string to get from and to keys
    initial = parse.urlsplit(url)
    parsed = parse.parse_qs(parse.urlsplit(url).query)
    dict_url = dict(parse.parse_qsl(parse.urlsplit(url).query))

    # Set new times
    now = datetime.datetime.now()
    before = now - datetime.timedelta(minutes=10)

    # Convert new times back to epoch
    from_epoch = round((before - datetime.datetime(1970,1,1)).total_seconds() * 1000)
    to_epoch = round((now - datetime.datetime(1970,1,1)).total_seconds() * 1000)

    # Replace old times at from and to keys with new times
    dict_url["from"] = from_epoch 
    dict_url["to"] = to_epoch

    # Re-encode url string and return it
    new_parsed = parse.urlencode(dict_url)
    new_initial = initial._replace(query = new_parsed)
    newurl = parse.urlunsplit(new_initial)
    
    return(newurl)

In [5]:
def get_day(url):
    # Split url string to get from and to keys
    initial = parse.urlsplit(url)
    parsed = parse.parse_qs(parse.urlsplit(url).query)
    dict_url = dict(parse.parse_qsl(parse.urlsplit(url).query))

    # Get value at date
    day = str(dict_url["date"])
    
    return(day)

In [6]:
# Get key, value dataframe from any-level-nested json
def loopfunc(y):
    # General list declared
    pairs = []
    def loop(x):
        # Enter dictionary and check type of value
        for key, val in x.items():
            
            # If value is dictionary, call function with the value as dictionary
            if isinstance(val, dict): 
                loop(val)
                
            # If value is iterable, go into each value in iterable
            elif isinstance(val, list) or isinstance(val, set):
                for subval in val:
                    
                    # If value is dictionary, loop function with value as dictionary
                    if isinstance(subval, dict): 
                        loop(subval)
                    
                    # If no, append key and value
                    else:
                        pairs.append([key, subval])
            
            # If value is a single item and non-dictionary, append key and value
            else:
                pairs.append([key, val])
    loop(y)
    
    # Convert general list to dataframe and return dataframe 
    flat_json = pd.DataFrame(pairs, columns=["column", "value"])
    
    return(flat_json)

In [7]:
# Read input, prepare data 
def source_import():  
    source = input("Source data link: ")  
    
    # Update url if necessary
    # url_obj = urlupdate(source)

    # Update url, open and load into dictionary and return dataframe
    response = requests.get(source)
    file = response.json() 
    data = loopfunc(file)
    
    # Specific for Strom
    # Get date from url
    day = get_day(source)
    
    # Remove non-alphanumeric characters from data keys, add underscore in spaces and make lowercase
    data["column"] = data["column"].str.split(".").str[-1].replace(r"([A-Z])", r" \1", 
                                                                    regex = True).replace("[^A-Za-z0-9 ]", "", regex=True)    
    data["column"] = data["column"].str.strip().str.lower()
    
    # Write company and day to dataframe
    data.loc[data.shape[0]] = ["company", "STROMDAO"]
    data.loc[data.shape[0]] = ["day", day]
    
    print("Data imported")
    return(data)

In [8]:
# Read ontology, prepare data
ontoin = input("Target ontology file: ")
onto = get_ontology(f"file://{ontoin}").load()

def onto_import(frame):
    # Remove ontology prefix from classes list
    ontolist = [str(i).split(".")[1] for i in list(onto.classes())]
    
    #* Get first instance from each class for data matcher and remove ontology name 
    #instance_list = [str(i).split(".")[1] for j in ontolist for i in 
                     #list(str(onto.search_one(iri = f"*{j}").instances())[0])]
    
    # Add lists into source frame
    frame["ontoclass"] = pd.Series(ontolist)
    
    #* If instance_list: frame["ontoclass"] = pd.Series(ontolist), frame["instance"] = pd.Series(instance_list)
    
    # Remove non-alpha characters from data keys, add underscore in spaces and make lowercase 
    frame["class"] = frame["ontoclass"].str.split(".").str[-1].replace(r"([A-Z])", r" \1", 
                                                                       regex = True).replace("[^A-Za-z0-9 ]", "", regex=True)
    frame["class"] = frame["class"].str.strip().str.lower()
    
    print("Ontology imported")
    return(frame)


Target ontology file: C:/Users/Kez/OneDrive/Documents/Jacobs/Yr 3/Thesis/Neo4j_Ontology/Strom


In [9]:
# Schema comparing function 
def schema_matcher(frame):
    
    # Data matcher (compares data types)
    def data_matcher(frame): 
        # Empty output list declared
        filter_list = []

        # Check if each value in source data is the same type as class instance
        if type(frame["value"]) == type(frame["instance"]):

            # If yes, append class name and data key 
            filter_list.append([frame["column"], frame["class"]])

        return(filter_list)
    
    # Semantic matcher
    def semantic_matcher(class_col_list): 
        # Empty output list created
        firstmatch = []
        
        # Declare class and column positions
        for item in class_col_list:
            var_column = item[0]
            var_class = item[1]
            semantic_match = 0
            
            # If the words are the same, similarity is 1.0
            if var_column == var_class:
                semantic_match = 1.0
            
            else:
                # Check if class and column are both single words 
                if " " not in var_column and " " not in var_class:

                    # If single words, then create wordnet instances
                    column_list = wn.synsets(var_column)
                    class_list = wn.synsets(var_class)

                    # Check if words exist in wordnet
                    if column_list and class_list:
                        col_synonym = column_list[0]
                        class_synonym = class_list[0]
                        semantic_match = round(col_synonym.wup_similarity(class_synonym), 2)

                    # If not, check word2vec for word and calculate word similarity
                    else:
                        # Handle events where variables not in model
                        if var_column in model.key_to_index and var_class in model.key_to_index:
                            semantic_match = round(model.similarity(var_column, var_class), 2)

                # If class and column are not both single words, use word2vec for phrase similarity 
                else:
                    #Handle trailing words
                    if var_column in var_class.split(" ") or var_class in var_column.split(" "):
                        semantic_match = 0.8
                    
                    else:
                        # Handle events where variables not in model
                        if var_column in model.key_to_index and var_class in model.key_to_index:
                            semantic_match = round(model.n_similarity(var_column, var_class), 2)
                   
            # If similarity passes benchmark score, append column, class and similarity score
            if semantic_match >= 0.7:
                firstmatch.append([var_column, var_class, semantic_match])
         
        # Make dataframe from list and return
        newframe = pd.DataFrame(firstmatch, columns = ["matched_column", "matched_class", "schema_score"])
        
        return(newframe)
    
    #* Call data matcher (to be used as semantic matcher input)    
    #class_cols = data_matcher(frame)
    
    # Instead of data matcher, get list of class-column pairs for semantic matcher input
    class_cols = [[i, j] for i in frame["column"].unique() for j in frame["class"] if i == i and j == j]
    
    # Call semantic matcher
    schema_score_frame = semantic_matcher(class_cols)  
    
    print("Schema matcher done")
    return(schema_score_frame) 

In [10]:
# Example comparing function
def example_matcher(frame):
    # Get object property list from ontology and prepare strings
    properties = [[re.sub("(?<!^)(?=[A-Z])", " ", str(i).split(".")[1]).lower(), i] 
                  for i in list(onto.object_properties())]
     
    # Initialise rdf graph 
    graph = rdflib.Graph()
    
    # Initialise spacy
    english = spacy.load("en_core_web_sm")
    english.Defaults.stop_words |= {"function","relates","reading"}
    stop_words_spacy = english.Defaults.stop_words
    
    frame["example_score"] = 0
    
    for property_pair in properties:
        # Get property string
        prop = property_pair[0]
        
        # Get range and prepare string
        ontoprop = re.split(r"[^a-zA-Z]", str(property_pair[1].range))[-2]
        
        # Get range and domain class of property
        rng = re.sub(r"([A-Z])", r" \1", ontoprop).strip().lower()
        
        # Extract keywords and re-join
        words = [word for word in prop.split() if not word in stop_words_spacy]
        keywords = "_".join(words).title()
        
        # Get rdf related to property
        uri = rdflib.URIRef(f"http://dbpedia.org/resource/{keywords}")
        graph.parse(uri)

        # Use property uri as predcicate 
        for pred, obj in graph.predicate_objects(uri):
            if isinstance(obj, rdflib.term.Literal) and obj.language == "en":
                
                #compare columns to objects
                for i in frame["matched_column"]:
                    example_score = round(model.n_similarity(i, obj), 2)
                
                     # If similarity passes benchmark score
                    if example_score >= 0.7:
                        # Assign scores in data dataframe to respective rows
                        frame.at[frame["matched_column"] == i & frame["matched_class"] == rng, 
                                 "example_score"] = example_score     

    # Calculate mean overall match score by row
    frame["match_total"] = np.where(frame["example_score"] != 0, 
                                    frame[["schema_score", "example_score"]].mean(axis=1), frame["schema_score"])
    
    print("Example matcher done")
    return(frame)


#### Sample input
Enter XML file: C:\Users\Kez\OneDrive\Documents\Jacobs\Yr 3\Thesis\Neo4j_Ontology\Strom_mapping.xml

In [11]:
# Map to ontology and add relations
relations = input("XML file: ")
tree = ET.parse(relations)
root = tree.getroot()

def mapper(frame, frame_two):     
    # Filter by overall match mean and handle duplicate matches 
    frame = frame[frame["match_total"] >= 0.7]
    frame = frame.sort_values("match_total", ascending=False).drop_duplicates(["matched_column", "matched_class"])
    frame_array= frame[["matched_column", "matched_class"]].to_numpy()
    
    # Declare list of matches
    matches = []
    
    # Iterate through the dataframe, get respective values
    for pair in frame_array:
        # Get ontology class in onto dataframe, where refined class is same as class in frame array 
        frameclass = frame_two.loc[frame_two["class"] == pair[1], "ontoclass"].values[0]
        
        # Get data values in onto dataframe, where refined column name is same as column in frame array 
        actual_column = frame_two.loc[frame_two["column"] == pair[0], "column"].values[0]
        values = frame_two.loc[frame_two["column"] == pair[0], "value"]
        
        # Append matches to list
        matches.append([actual_column, frameclass])
        
        # Call the class in the ontology
        ontoclass = onto.search_one(iri = f"*{frameclass}")
        
        # instantiate count of instances for each class
        count = 0
        
        # Write values in ontology
        for value in values:
            ind = ontoclass(f"{value}")
    
            # Iterate through elements in root 
            for elem in root.iter("mapping-element"):
                # Get domain and range classes, and property
                propertydomain = elem[0].text
                propertyrange = elem[1].text 
                relation = elem[2].text

                # Match newly-written instance to element tree classes
                if frameclass == propertydomain:
                    # If same as domain, find range class in ontology
                    rangeclass = onto.search_one(iri = f"*{propertyrange}")

                    # Get corresponding class for property range
                    onto_rng = frame_two.loc[frame_two["ontoclass"] == propertyrange, "class"].values[0]

                    # Get corresponding matched column for class
                    match_rng = frame.loc[frame["matched_class"] == onto_rng, "matched_column"].values[0]

                    # Get value at nth occurrence of column 
                    try:
                        rng_value = frame_two.loc[frame_two["column"] == match_rng, "value"].values[count]
                    except IndexError:
                        rng_value = frame_two.loc[frame_two["column"] == match_rng, "value"].values[0]
                    
                    # Write to ontology
                    rng_ind = rangeclass(f"{rng_value}")

                    # Create relation
                    ind.relation = [rng_ind]
                
                elif frameclass == propertyrange:
                    # If same as range, find domain class in ontology
                    domainclass = onto.search_one(iri = f"*{propertydomain}")

                    # Get corresponding class for property domain
                    onto_dom = frame_two.loc[frame_two["ontoclass"] == propertydomain, "class"].values[0]

                    # Get corresponding matched column for class
                    match_dom = frame.loc[frame["matched_class"] == onto_dom, "matched_column"].values[0]
                    
                    # Get value at nth occurrence of column
                    try:
                        dom_value = frame_two.loc[frame_two["column"] == match_dom, "value"].values[count]
                    except IndexError:
                        dom_value = frame_two.loc[frame_two["column"] == match_dom, "value"].values[0]
                    
                    # Write to ontology
                    dom_ind = domainclass(f"{dom_value}")

                    # Create relation
                    dom_ind.relation = [ind]
            
            count += 1    
    
    # Convert list to array
    matches_array = np.array(matches)
    
    print("Mapping done")   
    return(matches)


XML file: C:\Users\Kez\OneDrive\Documents\Jacobs\Yr 3\Thesis\Neo4j_Ontology\Strom_mapping.xml


In [12]:
# Save in Neo4j function (Neo4j Desktop must be open beforehand)
def map_import():
    
    # Class for importing file into Neo4j
    class AutoImport:
        # Initialise session
        def __init__(self, uri, user, password):
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
        # Close session
        def close(self):
            self.driver.close()
        # Import file and run session
        def autoimport(self, file):
            with self.driver.session() as session:
                f = f'CALL n10s.rdf.import.fetch("file:///{file}","RDF/XML");'
                imp = session.run(f)
    
    # Get output ontology file and save new data to file
    output = root.find("output").text
    onto.save(file = f"{output}")

    try:
        # Import into Neo4j
        test = AutoImport("bolt://localhost:7687", "neo4j", "password")
        
        # Prepare output string for double backslach and no .extension
        import_file = output.replace("/", "\\")
        
        test.autoimport(import_file) 
        test.close()
        print("Importing done, wait a few minutes for Neo4j to display")

    except Exception as V:
        print(V)

In [13]:
# Calling all functions
def main():
    data_frame = source_import()

    onto_frame = onto_import(data_frame)

    schema_frame = schema_matcher(onto_frame)

    example_frame = example_matcher(schema_frame)

    actual_array = mapper(example_frame, onto_frame)

    map_import()
    
    return(actual_array)

#### Sample input
Source data: https://api.corrently.io/v2.0/delfine/dsi?zip=69168&date=2021-12-12

*big data: https://api.corrently.io/v2.0/datadomain/query?db=openems&fields=meter0/ActivePower,meter1/ActivePower,meter2/ActivePower,_sum/ConsumptionActivePower&from=1635951600000&to=1635980400000&edge=1237

In [14]:
# Profiling performance with snakeviz for visualization
%load_ext snakeviz

# Calling main function to return actual matches array
%snakeviz actual_array = main()

Source data link: https://api.corrently.io/v2.0/delfine/dsi?zip=69168&date=2021-12-12
Data imported
Ontology imported
Schema matcher done
Example matcher done
Mapping done
Importing done, wait a few minutes for Neo4j to display
 
*** Profile stats marshalled to file 'C:\\Users\\Kez\\AppData\\Local\\Temp\\tmpvl_k1flx'. 
Embedding SnakeViz in this document...


#### Sample input
True matches: C:\Users\Kez\OneDrive\Documents\Jacobs\Yr 3\Thesis\Neo4j_Ontology\Strom_true.txt

In [15]:
# Profiling accuracy for test data
# Loading true matches array
text_file = input("True matches text file: ")
true_array = np.loadtxt(text_file, dtype=str)

# Convert labels to binary for f-score calculations 
mlb = MultiLabelBinarizer()
true_match = mlb.fit_transform(true_array)
actual_match = mlb.fit_transform(actual_array)

# Calculate metrics
p = precision_score(true_match, actual_match, average="macro")
r = recall_score(true_match, actual_match, average="macro")
f = f1_score(true_match, actual_match, average="macro")

print("Result: p=%.3f, r=%.3f, f=%.3f" % (p, r, f))

True matches text file: C:\Users\Kez\OneDrive\Documents\Jacobs\Yr 3\Thesis\Neo4j_Ontology\Strom_true.txt
Result: p=1.000, r=1.000, f=1.000
